<a href="https://colab.research.google.com/github/ttobio/VS/blob/main/ligand_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Preparing chemical structures
##This code goes through these steps
1. Reading a list of smiles (in a csv file)
2. Generating 2d structures
3. Generating 3D structures and perform energy minimizataion
4. adding hydrognes (all hydrogens)
5. Calculating charges through the InChI output

In [1]:
# Download and install Open Babel (This will take a couple of minutes)
!wget -q https://github.com/openbabel/openbabel/archive/refs/tags/openbabel-3-1-1.tar.gz
!tar -xzf openbabel-3-1-1.tar.gz
%cd openbabel-openbabel-3-1-1
!mkdir build
%cd build
!cmake ..
!make
!make install
!ldconfig
%cd ../..

!pip install rdkit

/content/openbabel-openbabel-3-1-1
/content/openbabel-openbabel-3-1-1/build
CMake Deprecation Warning at CMakeLists.txt:3 (cmake_minimum_required):
  Compatibility with CMake < 3.5 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
CMake Deprecation Warning at CMakeLists.txt:14 (cmake_policy):
  The OLD behavior for policy CMP0042 will be rem

In [3]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
import os

In [4]:
# 1. Read CSV file with two columns: id and smiles
csv_file = '/content/data/z_gp_ligands.csv'  # replace with your actual CSV file path
data = pd.read_csv(csv_file)


In [5]:
# 2. Generate 2D SDF files for each compound
sdf_dir = "/content/data/sdf"
os.makedirs(sdf_dir, exist_ok=True)

for index, row in data.iterrows(): #The csv file must contain two columns: id, smiles
    compound_id = row['id']
    smiles = row['smiles']

    # Create RDKit molecule from SMILES
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        # Generate 2D coordinates
        AllChem.Compute2DCoords(mol)

        # Save to SDF
        sdf_path = os.path.join(sdf_dir, f"{compound_id}_2d.sdf")
        w = Chem.SDWriter(sdf_path)
        w.write(mol)
        w.close()


In [6]:
# 3. Minimize the energy using OpenBabel (obgen)
for sdf_file in os.listdir(sdf_dir):
    if sdf_file.endswith("_2d.sdf"):
        sdf_path = os.path.join(sdf_dir, sdf_file)
        base_name = sdf_file.replace("_2d.sdf", "")

        # Perform geometry optimization and save 3D structure
        os.system(f"obgen {sdf_path} -ff MMFF94 > {os.path.join(sdf_dir, base_name + '_3d.sdf')}") #you can change the force field after -ff
        print(f"Processed {sdf_file} -> {base_name}_3d.sdf")

Processed AS-4_2d.sdf -> AS-4_3d.sdf
Processed AS-3_2d.sdf -> AS-3_3d.sdf
Processed AS-5_2d.sdf -> AS-5_3d.sdf
Processed AS-6_2d.sdf -> AS-6_3d.sdf
Processed AS-85_2d.sdf -> AS-85_3d.sdf
Processed AS-99_2d.sdf -> AS-99_3d.sdf
Processed AS-1_2d.sdf -> AS-1_3d.sdf
Processed AS-nc_2d.sdf -> AS-nc_3d.sdf
Processed AS-2_2d.sdf -> AS-2_3d.sdf


In [8]:
# 4. Add all hydrogens to the 3D structure and calculate the charge
for sdf_file in os.listdir(sdf_dir):
    if sdf_file.endswith("_3d.sdf"):
        sdf_path = os.path.join(sdf_dir, sdf_file)

        # Add hydrogens and overwrite the file with hydrogens added
        os.system(f"obabel {sdf_path} -O {sdf_path.replace('.sdf', '_h.sdf')} -h")

        # Calculate the charge of the molecule
        charge_output = os.popen(f"obabel {sdf_path.replace('.sdf', '_h.sdf')} -oinchi").read()

        # Extract the charge from the InChI output (InChI string contains charge information)
        if "/q" in charge_output:
            charge = charge_output.split("/q")[-1].strip().split("/")[0]
        else:
            charge = 0  # Default to 0 if no charge is mentioned

        print(f"Compound {sdf_file} has charge: {charge}")

print("Process completed!")


Compound AS-2_3d.sdf has charge: 0
Compound AS-4_3d.sdf has charge: 0
Compound AS-1_3d.sdf has charge: 0
Compound AS-nc_3d.sdf has charge: 0
Compound AS-99_3d.sdf has charge: 0
Compound AS-3_3d.sdf has charge: 0
Compound AS-5_3d.sdf has charge: 0
Compound AS-6_3d.sdf has charge: 0
Compound AS-85_3d.sdf has charge: 0
Process completed!


In [9]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [10]:
import shutil

# Define your folder path and destination in Drive
folder_path = "/content/data/sdf"
drive_path = "/content/drive/My Drive/sdf_results"  # This will be the folder in your Drive

# Copy the folder to Drive
shutil.copytree(folder_path, drive_path)

print(f"Folder '{folder_path}' successfully copied to Google Drive at '{drive_path}'")


Folder '/content/data/sdf' successfully copied to Google Drive at '/content/drive/My Drive/sdf_results'
